<a href="https://colab.research.google.com/github/sobti/TSAI/blob/master/webcam/webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab import output
from base64 import b64decode
import base64
import time
import sys
import numpy as np
import cv2
import logging
#from google.colab import output
from PIL import Image
from io import BytesIO
from google.colab.patches import cv2_imshow

In [16]:
def data_uri_to_img(uri,i):
    
    """convert base64 image to numpy array"""
    try:
        filename='image' + str(i) + ".jpg"
        print(filename)
        image = base64.b64decode(uri.split(',')[1], validate=True)
        with open(filename, 'wb') as f:
         f.write(image)
        # make the binary image, a PIL image
        image = Image.open(BytesIO(image))
        # convert to numpy array
        image = np.array(image, dtype=np.uint8); 
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        cv2_imshow(gray)
        cv2.destroyAllWindows()
        return image
    except Exception as e:
        logging.exception(e);print('\n')
        return None



In [5]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      //const capture = document.createElement('button');
      //capture.textContent = 'Capture';
      //div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);
      //await stream.getTracks().forEach(track => track.stop())
      var i = 0;
      while (video) {
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        img= canvas.toDataURL('image/jpeg', quality);
        google.colab.kernel.invokeFunction('notebook.data_uri_to_img', [img,i], {})
        await new Promise(resolve => setTimeout(resolve, 250));
        i=i+1
        }
      stream.getVideoTracks()[0].stop(); // stop video stream  
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  print("ll")
  #binary = b64decode(data.split(',')[1])
  #with open(filename, 'wb') as f:
  #  f.write(binary)
  #return filename

In [ ]:
# register this function, so JS code could call this
output.register_callback('notebook.data_uri_to_img', data_uri_to_img)

try:
  take_photo()

except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))